In [50]:
import pandas as pd
import numpy as np

from script import cleaner

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [51]:
train = pd.read_csv("./data/train.csv")
store = pd.read_csv("./data/store.csv")

/tmp/ipykernel_293577/2238278385.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("./data/train.csv")


In [68]:
df = cleaner(train, store)

In [69]:
df[df['Sales']==0]

,Date,DayOfWeek,Sales,Customers,Promo,StoreType,Assortment,CompetitionDistance,Promo2,PromoInterval,Competition_Since_X_months,weeks_since_promo2
52,2013-03-16,6,0,0,0,b,b,900.0,1,2,0,535
30431,2014-07-24,4,0,0,0,a,a,15430.0,1,2,128,635
40404,2013-01-30,3,0,0,0,a,c,2280.0,1,1,0,539
46576,2014-03-12,3,0,0,0,c,c,1390.0,0,0,223,0
99520,2014-07-12,6,0,0,0,a,a,150.0,1,1,187,487
99521,2014-07-24,4,0,0,0,a,a,150.0,1,1,187,487
127248,2014-07-24,4,0,0,0,a,a,320.0,1,2,0,712
128010,2013-01-24,4,0,0,1,c,c,13570.0,1,1,158,539
169888,2013-08-08,4,0,0,0,a,a,480.0,0,0,0,0
170457,2014-07-03,4,0,0,1,c,a,470.0,1,3,196,600


In [52]:
X = df.drop(columns=['Sales'])
y = df.Sales

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the column transformer for different preprocessing steps
numeric_features = ['Competition_Since_X_months', 'weeks_since_promo2', 'Customers', 'CompetitionDistance']
one_hot_features = ['Promo', 'Promo2','DayOfWeek',  'StoreType', 'Assortment', 'PromoInterval']


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

one_hot_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())
])


preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('ohe', one_hot_transformer, one_hot_features)
])

In [9]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth=10, min_samples_split=2, n_estimators=150)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', rf) ])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Competition_Since_X_months',
                                                   'weeks_since_promo2',
                                                   'Customers',
                                                   'CompetitionDistance']),
                                                 ('ohe',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Promo', 'Promo2',
                                                   'DayOfWeek', 'StoreType',
                                                   'Assortment',
                                                   'PromoInterval'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=10, n_estimators=150))])

In [10]:
# Evaluate the model on the test data
score = pipeline.score(X_test, y_test)

print(f"Random Forest - R-squared on test data:", score)

RAndom Forest - R-squared on test data: 0.9004734920784458


In [11]:
# Get predictions for the test data
y_pred = pipeline.predict(X_test)

In [29]:
y_test.isnull().sum()
np.isnan(y_pred).any()

y_pred.shape
y_test.shape

(82799,)

In [30]:
import numpy as np

def metric(preds, actuals):
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

In [32]:
rmspe = metric(y_pred, y_test.values)
rmspe

/tmp/ipykernel_293577/1119033358.py:5: RuntimeWarning: divide by zero encountered in divide
  return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])
/tmp/ipykernel_293577/1119033358.py:5: RuntimeWarning: invalid value encountered in divide
  return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])


nan